In [1]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer
from transformers.modeling_bert import BertForMaskedLM

In [2]:
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
model = AutoModelWithLMHead.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

In [3]:
input_ids = tokenizer.encode(f'''
    青葉山で{tokenizer.mask_token}の研究をしています。
''', return_tensors='pt')
print(input_ids)
print(tokenizer.convert_ids_to_tokens(input_ids[0].tolist()))

tensor([[    2,   872,  2311,   309,    12,     4,     5,   401, 13423,    11,
            46, 28457, 11665,     8,     3]])
['[CLS]', '青', '葉', '山', 'で', '[MASK]', 'の', '研', '究', 'を', '##して', '##い', '##ます', '。', '[SEP]']


In [4]:
masked_index = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
print(masked_index)

5


In [5]:
result = model(input_ids)
pred_ids = result[0][:, masked_index].topk(5).indices.tolist()[0]
for pred_id in pred_ids:
    output_ids = input_ids.tolist()[0]
    output_ids[masked_index] = pred_id
    print(tokenizer.decode(output_ids))

[CLS] 青 葉 山 で 学問 の 研 究 をしています 。 [SEP]
[CLS] 青 葉 山 で 武術 の 研 究 をしています 。 [SEP]
[CLS] 青 葉 山 で 武芸 の 研 究 をしています 。 [SEP]
[CLS] 青 葉 山 で 修行 の 研 究 をしています 。 [SEP]
[CLS] 青 葉 山 で 剣術 の 研 究 をしています 。 [SEP]


In [10]:
from transformers import BertTokenizer, AlbertForMaskedLM
import torch
from torch.nn.functional import softmax

pretrained = 'voidful/albert_chinese_base'
tokenizer = BertTokenizer.from_pretrained(pretrained)
model = AlbertForMaskedLM.from_pretrained(pretrained)

inputtext = "今天[MASK]情很好"

maskpos = tokenizer.encode(inputtext, add_special_tokens=True).index(103)

input_ids = torch.tensor(tokenizer.encode(inputtext, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids, masked_lm_labels=input_ids)

loss, prediction_scores = outputs[:2]
logit_prob = softmax(prediction_scores[0, maskpos]).data.tolist()
predicted_index = torch.argmax(prediction_scores[0, maskpos]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token,logit_prob[predicted_index])

感 0.3632276654243469


/Users/xiaofeiwu/miniconda3/envs/bigdata/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [11]:
predicted_tokens = tokenizer.convert_ids_to_tokens([predicted_index])
for predicted_token in predicted_tokens:
    print(predicted_token,logit_prob[predicted_index])

感 0.3632276654243469


In [15]:
from transformers import pipeline
nlp=pipeline('fill-mask', model=model, tokenizer=tokenizer)
nlp('今天[MASK]情很好')

[{'sequence': '[CLS] 今 天 感 情 很 好 [SEP]',
  'score': 0.3632276654243469,
  'token': 2697},
 {'sequence': '[CLS] 今 天 缘 情 很 好 [SEP]',
  'score': 0.053004853427410126,
  'token': 5357},
 {'sequence': '[CLS] 今 天 好 情 很 好 [SEP]',
  'score': 0.041060544550418854,
  'token': 1962},
 {'sequence': '[CLS] 今 天 人 情 很 好 [SEP]',
  'score': 0.0362703762948513,
  'token': 782},
 {'sequence': '[CLS] 今 天 恋 情 很 好 [SEP]',
  'score': 0.03572859615087509,
  'token': 2605}]